### Login

In [ ]:
import os
import requests

BASE_URL = 'http://localhost:1234'

def write_token(token):
    if not os.path.exists('token.txt'):
        with open('token.txt', 'w') as f:
            f.write(token)
    else:
        with open('token.txt', 'w') as f:
            f.write(token)

def login(user_name, pwd):
	response = requests.post(
		f'{BASE_URL}/users/login',
		json={
			'user_name': user_name,
			'pwd': pwd,
		}
	)
	res = response.json()
	token = res.get('data', {})
	if token:
		write_token(token)
	return res
login(user_name='', pwd='')

## Def

In [ ]:
import json
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

BASE_URL = 'http://localhost:1234'

def read_token(file_path='./token.txt'):
    try:
        with open(file_path, 'r') as file:
            return file.read().strip()
    except IOError as err:
        print(f'Error reading token from file: {err}')
        return None
    

def get_headers(token):
    return {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

token = read_token()

def _make_get_request(endpoint, params=None, quiet=None):
    """共用的 HTTP GET 請求處理函數"""
    try:
        response = requests.get(f"{BASE_URL}/{endpoint}", headers=get_headers(token), params=params)
        response.raise_for_status()
        result = response.json()
        if not quiet: 
            print(result)
        return result
    except requests.exceptions.RequestException as err:
        print(f"Request failed: {err}")
        return None
    
def _make_post_request(endpoint, data, quiet=None):
	try:
		response = requests.post(f"{BASE_URL}/{endpoint}", headers=get_headers(token), json=data)
		response.raise_for_status()
		result = response.json()
		if not quiet: 
			print(result)
		return result
	except requests.exceptions.RequestException as err:
		print(f"Request failed: {err}")
		return None

def register(user_name, pwd, email):
	res = _make_post_request('/users/register', {
			'user_name': user_name,
			'pwd': pwd,
			'email': email,
	})
	data = res.get('data', {})
	if data:
		token = data.get('token')
		if token:
			write_token(token)
	return res

            
def get_all_momentum():
    res = _make_get_request('market/momentum')
    return res.get('data')

def get_momentum_by_range(days):
    res = _make_get_request(f'market/momentum/range/{days}')
    return res.get('data')
    
def get_market_weights():
    res = _make_get_request('market/weights')
    return res.get('data')

def get_stock_prices():
    res = _make_get_request('market/stock/prices')
    return res.get('data')

def get_stock_symbols():
    res = _make_get_request('market/stock/symbols')
    return res.get('data')

def get_market_breadth():
    res = _make_get_request('market/stock/breadth')
    return res.get('data')

def get_stock_winners():
    res = _make_get_request('market/stock/winners')
    return res.get('data')

def get_stock_losers():
    res = _make_get_request('market/stock/losers')
    return res.get('data')

def clone_db_transactions():
    res = _make_get_request('transactions/clone')
    return res

def get_all_transactions():
    res = _make_get_request('transactions')
    return res


def parse_date(date_str):
    date_format = '%Y-%m-%d'
    try:
        date = datetime.strptime(date_str, date_format)
        return date.strftime(date_format)
    except ValueError:
        return None

def record_my_transactions(transactions):
    try:
        for stock_id, transaction_type, quantity, price, transaction_date in transactions:
            data = {
                'stock_id': stock_id,
                'transaction_type': transaction_type,
                'quantity': quantity,
                'price': price,
                'transaction_date': parse_date(transaction_date) or datetime.now().strftime('%Y-%m-%d')
            }
            response = requests.post(f'{BASE_URL}/transactions', headers=get_headers(token), json=data)
            if response.status_code in range(200, 300):
                print(f'Successfully inserted: {data}')
            else:
                print(f'Failed to insert: {data}, Status code: {response.status_code}, Response: {response.text}')
            time.sleep(0.5)
    except requests.exceptions.RequestException as err:
        print(err)

def get_my_portfolio():
    res = _make_get_request('portfolio')
    return res.get('data')

def update_my_portfolio(params):
    res = _make_post_request('portfolio', params)
    return res

def get_technews_by_keyword(keyword):
    res =  _make_get_request(f"technews/search", params={'keyword': keyword}, quiet=True)
    return res.get('data')[::-1]
    
def get_all_technews (page, size):
	res =  _make_get_request('technews', {
		'page': page,
		'size': size
	})
	return res.get('data')[::-1]
    
def create_one_company_news (params):
    res = _make_post_request('/company-news', params)
    return res
    
def get_all_company_news (page, size):
	res =  _make_get_request('company-news', {
		'page': page,
		'size': size
	})
	return res.get('data')[::-1]
    
def get_all_company_news_by_keyword (keyword, page=1, size=10):
	res =  _make_get_request('/company-news/query', {
		'page': page,
		'size': size,
		'keyword': keyword
	})
	return res.get('data')

def get_statement_by_symbol(symbol):
    return _make_get_request(f"statements/{symbol}")
    
def get_user_favorite_news():
    res = _make_get_request('userFavorite/technews')
    return res.get('data')
    
def user_subscribe_news(params):
    res = _make_post_request('/userFavorite/technews', params)
    return res

def post_comment (params):
    res = _make_post_request('comment', params)
    return res

def get_comment_by_post_id (id, page=1, size=10):
    res = _make_get_request(f"comment/post/{id}", {
		'page': page,
		'size': size
	})
    return res

def update_comment (params):
    res = _make_post_request('comment', params)
    return res
    
def print_news(news):
    for item in news:
        company_name = item.get('companyName', '')
        news_id = item.get('id', '--')
        title = item.get('title', '--')
        release_time = item.get('release_time', '--')
        web_url = item.get('web_url', '--')
        print(company_name)
        print(f"{news_id} : {title} {release_time} \n")
        if web_url:
            print(web_url)
        print('----------')
        

def draw_pie_chart(obj):
	labels = obj.keys()
	sizes = obj.values()
	colors = plt.get_cmap('Set2').colors
	explode = [0.1] * len(labels)

	plt.figure(figsize=(5, 5))
	plt.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', colors=colors, startangle=140)
	plt.title('Portfolio Distribution by Stock')
	plt.show()


def draw_line_chart(x_data, y_data, label, title, xlabel, ylabel, interval=10, figsize=(10, 6), color='#efb441'):
    plt.figure(figsize=figsize)
    plt.plot(x_data, y_data, marker='', linestyle='-', color=color, label=label)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))
    plt.gcf().autofmt_xdate()
    plt.grid(True)

    plt.legend()

    plt.tight_layout()
    plt.show()
    
def plot_pe_forward_vs_time(data):
    timestamps = []
    pe_forward_values = []
    
    for entry in data:
        created_at = entry.get('createdAt', '')
        pe_forward = entry.get('PE_Forward', None)
        
        if created_at and pe_forward is not None:
            timestamp = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%S.%fZ")
            timestamps.append(timestamp)
            
            pe_forward_values.append(float(pe_forward))
    
    draw_line_chart(timestamps, pe_forward_values, label='PE Forward', title='PE Forward vs Time', xlabel='Date', ylabel='PE Forward', figsize=(7, 5))
    



#### 註冊

In [ ]:
register(user_name='', pwd='', email='')

In [ ]:
res = get_market_weights()
res

#### 市場廣度指標

In [ ]:
breadth = get_market_breadth()
breadth

### 取得動能指標

In [ ]:
# my_data = get_all_momentum()

my_data = get_momentum_by_range(7)
print(my_data[0])
print(my_data[1])
print(my_data[2])
print(my_data[3])
print(my_data[4])
print(my_data[5])

#### 畫出動能

In [ ]:

import pandas as pd

df = pd.DataFrame(my_data)

df['createdAt'] = pd.to_datetime(df['createdAt'])
draw_line_chart(df['createdAt'], df['volume'], label='Momentum', title='Time Series Momentum', xlabel='Time', ylabel='Volume', interval=1)

In [ ]:
transactions = [
	# ('MU', 'sell', 41, 98.04, '2025-05-19'),
]

In [ ]:
record_my_transactions(transactions)
transactions = []

In [ ]:
symbols = get_stock_symbols()
symbols

In [ ]:
from decimal import Decimal, ROUND_HALF_UP

my_portfolio = get_my_portfolio()
stock_prices = get_stock_prices()
current_price_map = {item['symbol']: item['price'] for item in stock_prices}

PORTFOLIO_USD = 16478 + 6209.86

def validate_portfolio_item(item):
    required_keys = ['stock_id', 'quantity', 'average_price']
    if not all(key in item for key in required_keys):
        raise ValueError(f"Invalid portfolio item: {item}")
    try:
        quantity = float(item['quantity'])
        average_price = float(item['average_price'])
        if quantity < 0 or average_price < 0:
            raise ValueError(f"Negative quantity or price in item: {item}")
    except (ValueError, TypeError):
        raise ValueError(f"Invalid data type in item: {item}")
    
def process_portfolio(portfolio_dict, usd_value):
    portfolio_dict['USD'] = usd_value
    return dict(sorted(
        {k: v for k, v in portfolio_dict.items() if v > 0}.items(),
        key=lambda item: item[1],
        reverse=True
    ))

portfolio_values = {}
portfolio_with_current_price = {}

for item in my_portfolio:
    validate_portfolio_item(item)
    stock_id = item['stock_id']
    quantity = Decimal(str(item['quantity']))
    average_price = Decimal(str(item['average_price']))
    current_price = Decimal(str(current_price_map.get(stock_id, item['average_price'])))
    
    # 計算成本價值
    total_cost = (quantity * average_price).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
    portfolio_values[stock_id] = portfolio_values.get(stock_id, 0) + float(total_cost)
    
    # 計算當前市場價值
    total_current = (quantity * current_price).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
    portfolio_with_current_price[stock_id] = portfolio_with_current_price.get(stock_id, 0) + float(total_current)
    
    print(f"{stock_id}: {float(quantity):>6.2f} * ${float(average_price):>8.2f} = ${float(total_cost):>10.2f}")

# 處理和排序投資組合
sorted_portfolio_values = process_portfolio(portfolio_values, PORTFOLIO_USD)
sorted_current_values = process_portfolio(portfolio_with_current_price, PORTFOLIO_USD)

print('成本：')
draw_pie_chart(sorted_portfolio_values)
print('現在價位：')
draw_pie_chart(sorted_current_values)

In [ ]:

data = {
	'stock_id': 'TSLA',
	'average_price': 348,
	'quantity': 9
}
update_my_portfolio(data)

### search news by keyword

In [ ]:
keyword = 'ai'

news = get_technews_by_keyword(keyword)

print_news(news)

In [ ]:
all_news = get_all_technews(page=1, size=100)
print_news(all_news)

#### 搜索Company news

In [ ]:
all_news = get_all_company_news(page=1, size=100)

print_news(all_news)

In [ ]:
keyword = 'ai'
search_news = get_all_company_news_by_keyword(page=1, size=100, keyword=keyword)

print_news(search_news)

#### 讀取Html 轉成json資料

In [ ]:
symbol = ''

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime, timezone

with open('index.html', 'r', encoding='utf-8') as file:
	html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

news = []

for row in soup.find_all('tr', class_='cursor-pointer has-label'):
	time_cell = row.find('td', align='right')
	time_text = time_cell.get_text(strip=True).replace('Today', '')

	if 'Today' in time_text:
		time_text = time_text.replace('Today', '')
		now = datetime.now(timezone.utc)
		date_str = now.date().isoformat()
	else:
		now = datetime.now(timezone.utc)
		date_str = now.date().isoformat()

	try:
		time_obj = datetime.strptime(time_text, '%I:%M%p')
		time_24hr = time_obj.strftime('%H:%M:%S')
	except ValueError:
		# 如果时间格式无法解析，使用默认时间
		time_24hr = '00:00:00'
		
	# 创建 MySQL 支持的时间字符串
	release_time = f"{date_str} {time_24hr}"

	link = row.find('a', class_='tab-link-news')
	title = link.get_text(strip=True)
	web_url = link['href']

	publisher = row.find('span').get_text(strip=True).strip('()')

	news_item = {
        "title": title,
        "symbol": symbol,
        "release_time": release_time,
        "publisher": publisher,
        "web_url": web_url
    }

	news.append(news_item)

print(json.dumps(news[0:2], indent=4))


#### 單筆寫入

In [ ]:
from time import sleep

SLEEP_SECOND = 1 / 10
for item in news:
	res = create_one_company_news(item)
	print(res)
	sleep(SLEEP_SECOND)

#### GET statement

In [ ]:
res = get_statement_by_symbol('meta')
data = res.get('data')
plot_pe_forward_vs_time(data[-500:])

In [ ]:
# user_subscribe_news({'newsId': 116463})
data = get_user_favorite_news()
data

In [ ]:
clone_db_transactions()

In [ ]:
get_all_transactions()

In [ ]:
winners = get_stock_winners()
for stock in winners:
	print(stock.get('company'),'---', stock.get('dayChg'), '%')

In [ ]:
losers = get_stock_losers()
for stock in losers:
	print(stock.get('company'),'---', stock.get('dayChg'), '%')

In [ ]:
post_comment({
	'content': 'amd的晶圓消耗只有3%好嗎，nvda佔了77%',
    'postId': 121301,
})

In [ ]:
get_comment_by_post_id(121406)

In [ ]:
# post ID 和對應的中文假評論字典
from time import sleep

comments = {
    121397: "西洋棋王一人對抗14.3萬人還能和局？太強了！",
    121398: "量子科技和矽光子入列AI新十大建設？台灣科技野心不小！",
    121399: "台積電的CoWoS這麼先進，輝達沒其他選擇也正常！",
    121400: "金士頓在COMPUTEX展示AI伺服器和機器人，超期待！",
    121401: "MG銷售漸漸回溫，希望政府能幫忙解決關稅問題！",
    121402: "作業效率提升96%？AI生單助理真的太厲害了！",
    121403: "微軟的NLWeb讓網頁加AI超簡單，期待試用！",
    121404: "神雲在COMPUTEX展現三大主軸，後半年成長可期！",
    121405: "1.4萬年前的超級太陽風暴？地質史真神奇！",
    121406: "OpenAI花65億美元併購Jony Ive的公司？AI設計要大躍進！",
    121407: "GaN整合創新改變市場格局，供應鏈要動起來了！",
    121408: "Klarna用AI裁減40%人力？這效率也太誇張了！",
    121409: "選AI就像選員工要看價值觀？哈佛研究真有意思！",
    121410: "沃爾瑪裁員1500人？加快決策是關鍵吧！",
    121411: "AI面試官當機？這面試也太雷了吧！",
    121412: "目標百貨因關稅下修財測，供應鏈壓力真不小！",
    121413: "Meiko為蘋果訂單擴建越南廠？PCB需求爆棚！",
    121414: "泓德能源在澳洲投1.5GW，600億規模太震撼！",
    121415: "台積電發141億綠債，綠建築和再生能源有搞頭！",
    121416: "麗嬰房轉攻電商和物流，輕資產策略很聰明！",
    121417: "比特幣創歷史新高，保值潛力真的不容小覷！",
    121418: "Nike因關稅壓力要漲價？消費者要多掏錢了！",
    121419: "失竊iPhone流到深圳？這黑市也太誇張了吧！",
    121420: "黃仁勳家人看COMPUTEX，研華的智慧機器人超酷！",
    121421: "CoreWeave發債募20億美元，股價飆19%好猛！",
    121422: "信驊訂單熱絡，關稅和匯率影響沒那麼大！",
    121423: "Wolfspeed破產？台廠轉單機會來了！",
    121424: "iPhone 7 Plus和8成過時？蘋果更新真快！",
    121425: "美減稅法會不會重演英國三殺？川普要給答案了！",
    121426: "興達燃煤機組挨批，台電說符合環評還行嗎？",
    121427: "納維為NVIDIA開發新電源架構，股價飆200%太瘋狂！",
    121428: "SK海力士321層NAND Flash明年量產，技術真強！",
    121429: "駭客入侵川普幕僚通訊軟體？60個政府用戶中招？",
    121430: "台泥光電板下養羊？羊電共生計畫超有創意！",
    121431: "Hinge Health IPO定價32美元，遠距復健有前景！",
    121432: "NVIDIA GTC Taiwan展示AI醫療革命，馬偕合作很強！",
    121433: "俄羅斯把巴西變間諜工廠？這消息太震撼了！",
    121434: "納微為NVIDIA開發800V電力架構，股價狂漲180%！",
    121435: "川普精簡政府可能削弱地震監測？這風險不小！",
    121436: "微軟Aurora預測颶風比氣象專家還準？AI太強了！",
    121437: "昶瑞機電上櫃首日漲27.3%，中籤賺翻了！",
    121438: "放棄800億汽車貨物稅，到底誰最受益？",
    121439: "特斯拉Optimus會看影片？AI進步太恐怖了！",
    121440: "特斯拉Optimus突破AI新紀錄，未來可期！",
    121441: "元太電子紙因關稅產能吃緊，H5產線要加把勁！",
    121442: "0.1秒破解魔方？這機器人比人類眨眼還快！",
    121443: "Google I/O的AI產品功能重疊？有點混亂啊！",
    121444: "Zoom搭AI和混合辦公潮流，Q1財報超亮眼！",
    121445: "微軟批蘋果卡Xbox行動商店，政策爭議又來了！",
    121446: "川普承認普丁不想停戰？這局勢更複雜了！",
    121447: "咖啡生豆漲價？咖啡愛好者要哭了！",
    121448: "遠東銀推金融控股，AI還要再加強！",
    121449: "類星體用輻射束刺穿星系？宇宙太刺激了！",
    121450: "AI生成影片耗電等同微波爐一小時？太誇張！",
    121451: "斷電不是再生能源的錯，電網升級跟不上！",
    121452: "陽明交大高溫超導突破，量子糾纏太厲害！",
    121453: "財政惡化推高長債殖利率，債券市場要小心！",
    121454: "HBM4製造門檻高，溢價30%很合理！"
}

# 使用 for 迴圈遍歷 post ID 並呼叫 post_comment
for post_id, content in comments.items():
    post_comment({
        'content': content,
        'postId': post_id
    })
    sleep(0.25)